## Семинар №8: Регуляризация

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv('train.csv', index_col=0)
print(df.shape)
df.head()

(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Разобьем выборку на train и test

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

Предобработка

1) Заполнить пропуски

In [9]:
numeric_data = X_train.select_dtypes([np.number])

means = numeric_data.mean()
numeric_cols = numeric_data.columns

X_train = X_train[numeric_cols].fillna(means)
X_test = X_test[numeric_cols].fillna(means)

2) Масштабирование

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Обучаем модель

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [15]:
model = LinearRegression()

model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

In [17]:
for col, coef in zip(X_train.columns, model.coef_):
    print(f'{col} –– {coef}')

MSSubClass –– -8609.878378387213
LotFrontage –– -2979.129551623501
LotArea –– 4918.260012701947
OverallQual –– 24532.344025776347
OverallCond –– 4628.946226983778
YearBuilt –– 8326.937137014163
YearRemodAdd –– 3569.6045111099065
MasVnrArea –– 4264.24956164437
BsmtFinSF1 –– 1.296941777417342e+16
BsmtFinSF2 –– 4469872741767287.5
BsmtUnfSF –– 1.2610442885570806e+16
TotalBsmtSF –– -1.2449742442476568e+16
1stFlrSF –– -3.0143990616415868e+16
2ndFlrSF –– -3.4267565970991124e+16
LowQualFinSF –– -3733128506340379.0
GrLivArea –– 4.088312670210515e+16
BsmtFullBath –– 5850.580095885642
BsmtHalfBath –– -257.09963530249803
FullBath –– 1602.4488189124634
HalfBath –– -1147.0414611932308
BedroomAbvGr –– -6976.66265524089
KitchenAbvGr –– -2071.9300243683283
TotRmsAbvGrd –– 8234.400904784303
Fireplaces –– 3117.349186269324
GarageYrBlt –– 3089.4600360315326
GarageCars –– 9121.370585784574
GarageArea –– -886.7376573579618
WoodDeckSF –– 2808.3238712892557
OpenPorchSF –– -405.2205376435817
EnclosedPorch –– 5

In [19]:
mean_squared_error(y_train_pred, y_train) # ошибка на трейне

1150959101.8680248

In [22]:
mean_squared_error(y_test_pred, y_test) # ошибка на тесте

1356929220.3964906

Используем регуляризацию

In [23]:
from sklearn.linear_model import Ridge, Lasso

In [30]:
model = Ridge(alpha=162) # alpha это lambda

model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

In [31]:
for col, coef in zip(X_train.columns, model.coef_):
    print(f'{col} –– {coef}')

MSSubClass –– -6054.499445411721
LotFrontage –– -1550.6843498887267
LotArea –– 4180.648358032623
OverallQual –– 19909.032106847677
OverallCond –– 3714.6488344987906
YearBuilt –– 6066.327520591395
YearRemodAdd –– 5122.283620999754
MasVnrArea –– 4572.568984327124
BsmtFinSF1 –– 4352.744388252995
BsmtFinSF2 –– -347.711057603084
BsmtUnfSF –– 100.81090589190912
TotalBsmtSF –– 4511.708309432394
1stFlrSF –– 6497.50386867804
2ndFlrSF –– 7021.78273639413
LowQualFinSF –– 430.54842275023054
GrLivArea –– 10715.603886748197
BsmtFullBath –– 4928.464889701888
BsmtHalfBath –– -552.0823046645817
FullBath –– 3592.7911407716156
HalfBath –– 738.469491430182
BedroomAbvGr –– -4656.888012044249
KitchenAbvGr –– -3207.55919483965
TotRmsAbvGrd –– 7548.6550073121425
Fireplaces –– 4327.932733823072
GarageYrBlt –– 3256.8801631503934
GarageCars –– 7019.415537118618
GarageArea –– 2757.3464682190006
WoodDeckSF –– 3031.7854302000555
OpenPorchSF –– 111.87292683439223
EnclosedPorch –– 349.453877692602
3SsnPorch –– 1182.2